In [1]:
import gensim.downloader as api
wv=api.load("word2vec-google-news-300")

C:\Users\Amit\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
len(wv)

3000000

In [3]:
dir(wv)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_load_specials',
 '_log_evaluate_word_analogies',
 '_save_specials',
 '_smart_save',
 '_upconvert_old_d2vkv',
 '_upconvert_old_vocab',
 'add_lifecycle_event',
 'add_vector',
 'add_vectors',
 'allocate_vecattrs',
 'closer_than',
 'cosine_similarities',
 'distance',
 'distances',
 'doesnt_match',
 'evaluate_word_analogies',
 'evaluate_word_pairs',
 'expandos',
 'fill_norms',
 'get_index',
 'get_mean_vector',
 'get_normed_vectors',
 'get_vecattr',
 'get_vector',
 'has_index_for',
 'index2entity',
 'index2word',
 'index_to_key',
 'init_sims',
 

In [4]:
import spacy

In [5]:
nlp=spacy.load('en_core_web_lg')

In [6]:
import pandas as pd

In [7]:
df=pd.read_csv('Fake_Real_Data.csv')

In [8]:
df.shape

(9900, 2)

In [9]:
df.head(7)

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real
5,France says pressure needed to stop North Kore...,Real
6,"Trump on Twitter (August 8): Opioid crisis, No...",Real


In [10]:
df.label.value_counts()

Fake    5000
Real    4900
Name: label, dtype: int64

In [36]:
df['new_label']=df.label.map({'Fake':0,'Real':1})

In [12]:
def preprocess(text):
    doc=nlp(text)
    filtered_tokens=[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [13]:
def preprocess_vectorizer(text):
    doc=nlp(text)
    filtered_tokens=[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return wv.get_mean_vector(filtered_tokens)

In [14]:
df['nlp_text']=df.Text.apply(lambda text: nlp(preprocess(text)).vector)

In [15]:
df

,Text,label,new_label,nlp_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[0.23517725, 1.5265795, -2.0906563, 0.02504843..."
1,U.S. conservative leader optimistic of common ...,Real,0,"[-0.6691326, 0.62931126, -1.5525453, 1.4249555..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,0,"[-1.0787294, -0.42471677, -2.4816692, 1.496829..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-1.5941342, 0.4442709, -1.5567913, 2.0706172,..."
4,Democrats say Trump agrees to work on immigrat...,Real,0,"[-0.6243596, 0.7249644, -2.0244102, 1.5570543,..."
...,...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0,"[-0.19872437, 0.51496077, -1.3619812, -0.37650..."
9896,Trump consults Republican senators on Fed chie...,Real,0,"[-0.85058653, 0.37871653, -1.2622266, 1.412322..."
9897,Trump lawyers say judge lacks jurisdiction for...,Real,0,"[-0.39164215, 0.3089823, -1.316931, 1.2282809,..."
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0,"[-0.059299797, 0.6395275, -1.9863535, 0.518406..."


In [16]:
df['gensim_text']=df.Text.apply(lambda text: preprocess_vectorizer(text))

In [69]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [68]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.nlp_text,df.new_label,test_size=.2,random_state=2022,stratify=df.new_label)

In [25]:
import numpy as np

In [70]:
m=MinMaxScaler()

In [71]:
x_train_scl=m.fit_transform(x_train_stack)
x_test_scl=m.fit_transform(x_test_stack)

In [26]:
x_train_stack=np.stack(x_train)

In [30]:
x_test_stack=np.stack(x_test)

In [45]:
x_train[0][0]

0.23517725

In [46]:
x_test.shape

(1980,)

In [72]:
from sklearn.ensemble import RandomForestClassifier
R=RandomForestClassifier()
R.fit(x_train_scl,y_train)

RandomForestClassifier()

In [73]:
R.score(x_test_scl,y_test)

0.9717171717171718

In [75]:
y_pred=R.predict(x_test_scl)

In [33]:
from sklearn.metrics import confusion_matrix,classification_report

In [76]:
print(confusion_matrix(y_test,y_pred))

[[955  45]
 [ 11 969]]


In [77]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97      1000
           1       0.96      0.99      0.97       980

    accuracy                           0.97      1980
   macro avg       0.97      0.97      0.97      1980
weighted avg       0.97      0.97      0.97      1980



In [47]:
x_train,x_test,y_train,y_test=train_test_split(df.gensim_text,df.new_label,test_size=.2,random_state=2022,stratify=df.new_label)

In [48]:
x_train_stack=np.stack(x_train)
x_test_stack=np.stack(x_test)

In [49]:

R.fit(x_train_stack,y_train)
R.score(x_test_stack,y_test)

0.9671717171717171

In [51]:
y_pred=R.predict(x_test_stack)

In [52]:
print(confusion_matrix(y_test,y_pred))

[[957  43]
 [ 22 958]]


In [53]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1000
           1       0.96      0.98      0.97       980

    accuracy                           0.97      1980
   macro avg       0.97      0.97      0.97      1980
weighted avg       0.97      0.97      0.97      1980



In [91]:
x_train,x_test,y_train,y_test=train_test_split(df.Text,df.new_label,test_size=.2,random_state=2022,stratify=df.new_label)

x_train.shape

(7920,)

In [79]:
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('v',CountVectorizer()),
    ('m',RandomForestClassifier())
])

In [80]:
clf.fit(x_train,y_train)

Pipeline(steps=[('v', CountVectorizer()), ('m', RandomForestClassifier())])

In [81]:
clf.score(x_test,y_test)

0.997979797979798

In [82]:
y_pred=clf.predict(x_test)

In [83]:
y_pred[:4]

array([0, 1, 1, 0], dtype=int64)

In [84]:
y_test[:4]

8785    0
6331    1
6899    1
6405    0
Name: new_label, dtype: int64

In [85]:
df.loc[6331]

Text           White House budget chief says any tax package ...
label                                                       Real
new_label                                                      1
nlp_text       [-1.548101, -0.5154105, -0.9623005, 0.67930615...
gensim_text    [0.015165413, 0.004643604, -0.0007000496, 0.04...
Name: 6331, dtype: object

In [90]:
clf=Pipeline([
    ('v',TfidfVectorizer()),
    ('m',RandomForestClassifier())
])

In [93]:
clf.fit(x_train,y_train)

Pipeline(steps=[('v', TfidfVectorizer()), ('m', RandomForestClassifier())])

In [94]:
clf.score(x_test,y_test)

0.997979797979798

In [95]:
y_pred=clf.predict(x_test)

In [96]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000
           1       1.00      1.00      1.00       980

    accuracy                           1.00      1980
   macro avg       1.00      1.00      1.00      1980
weighted avg       1.00      1.00      1.00      1980



In [ ]:
print(confusion_matrix())